<a href="https://colab.research.google.com/github/rhaagsma/Projeto_Final_Marvel/blob/main/ProjetoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fazendo Importações e passando os parâmetros necessários para realizar as resquisições

In [ ]:
import hashlib
import time
import requests
from google.colab import userdata
import sqlite3
import pandas as pd
import ast

public_key = userdata.get('public')
private_key = userdata.get('private')

ts = str(time.time())
to_hash = ts + private_key + public_key
hash_digest = hashlib.md5(to_hash.encode()).hexdigest()
auth_params = {
        "ts": ts,
        "apikey": public_key,
        "hash": hash_digest,
        "offset": 0
    }

In [ ]:
url1 = "https://gateway.marvel.com/v1/public/characters"
url2 = "https://gateway.marvel.com/v1/public/comics"
url3 = "https://gateway.marvel.com/v1/public/events"

### Definição da Função que retorna os dados da Requisição em um Dataframe

In [ ]:
def get_data(url):
  total_data = 0
  offset = 0
  data = pd.DataFrame()

  # Garante a entrada no loop na primeira vez
  while offset < total_data or total_data == 0:
      print(offset)
      auth_params['offset'] = offset
      response = requests.get(url, auth_params)
      response = response.json()

      # Atualiza total_data após a primeira requisição
      if total_data == 0:
          total_data = response['data']['total']

      # Verifica se a resposta contém dados
      if not response['data']['results']:
          break

      response = pd.json_normalize(response['data']['results'])
      df1 = response
      data = pd.concat([data, df1])
      offset += len(df1)

  return data

### Definição da Função para criar uma tabela no banco

In [ ]:
def create_table(df, columns, name):
  conn = sqlite3.connect('banco.db')

  df[columns].to_sql(name, conn, if_exists='replace', index=False)

  conn.close()

### Requisição do Endpoint Characters e inserção dos dados no Banco

In [ ]:
characters = get_data(url1)

columns = ['id', 'name','description']
create_table(characters, columns, 'characters')

### Definição de uma Função que extrai os preços da lista de dicionários do Endpoint Comics

In [ ]:
def extrair_preco(lista_precos):
    try:
        # Converte a string em lista de dicionários, se necessário
        if isinstance(lista_precos, str):
            lista_precos = ast.literal_eval(lista_precos)

        for item in lista_precos:
            if item.get('type') == 'printPrice':
                return item.get('price')
    except Exception as e:
        print(f"Erro ao processar: {lista_precos} -> {e}")

    return 0


### Requisição do Endpoint Comics e inserção dos dados no Banco

In [ ]:
comics = get_data(url2)

# Expressão regular para encontrar um ano de 4 dígitos entre parênteses
comics['year'] = comics['title'].str.extract(r'\((\d{4})\)').fillna(0).astype(int)
comics['price'] = comics['prices'].apply(extrair_preco)

columns = ['id','digitalId','title','year','pageCount', 'price']
create_table(comics, columns, 'comics')

### Definição de uma função para associar entidades

In [ ]:
def associate_entities(main_entity, associated_entity, limit):
    conn = sqlite3.connect('banco.db')

    query = f"SELECT id FROM {main_entity}"
    df = pd.read_sql_query(query, conn)

    conn.close()

    request_count = 0
    data = pd.DataFrame()

    for index, row in df.iterrows():
        id = row['id']
        print(id)
        offset = 0
        total = 0

        while offset <= total or total == 0:
            params = auth_params
            params.update({
                'offset': offset,
                'limit': limit
            })

            api_resp = requests.get(f"https://gateway.marvel.com/v1/public/{main_entity}/{id}/{associated_entity}", params=params)
            request_count += 1

            response = api_resp.json()
            results = response['data']['results']

            temp_df = pd.DataFrame(results)

            temp_df[f'{main_entity}_id'] = id

            data = pd.concat([data, temp_df], ignore_index=True)

            if total == 0:
                total = response['data']['total']
                if total == 0:
                    break

            offset += limit

            request_count += 1
    return data

### Gerando dataframe de associação entre charactes e comics

In [ ]:
charactes_comics = associate_entities("characters","comics", 100)

comics_chars.rename(columns={'id': 'comic_id'})
columns = ['character_id', 'comic_id']
create_table(comics_chars, columns, 'characters_comics')

### Requisição do Endpoint Events e inserção dos dados no Banco

In [ ]:
events = get_data(url3)
create_table(events, ['id', 'title','description', 'start', 'end'], 'events')

### Gerando dataframe de associação entre events e comics

In [ ]:
comics_events = associate_entities("events", "comics", 20)

comics_events = comics_events.rename(columns={
    'id': 'comic_id',
    'events_id': 'event_id'
})

columns = ['comic_id', 'event_id']

create_table(comics_events, columns, 'comics_events')